In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
test =  pd.read_csv("/kaggle/input/restaurant-revenue-prediction/test.csv.zip")

In [ ]:
#目的変数を抽出
revenue = train["revenue"]

#前処理がしやすい様に、trainとtestを結合
df_whole = pd.concat([train, test], axis=0)
del df_whole["revenue"]
#Open Dateを年・月・日に分解
df_whole["Open Date"] = pd.to_datetime(df_whole["Open Date"])
df_whole["Year"] = df_whole["Open Date"].apply(lambda x:x.year)
df_whole["Month"] = df_whole["Open Date"].apply(lambda x:x.month)
df_whole["Day"] = df_whole["Open Date"].apply(lambda x:x.day)

#Cityを数値に変換
le = LabelEncoder()
df_whole["City"] = le.fit_transform(df_whole["City"])

# City Groupを数値に変換 Other -> 0, Big Cities -> 1
df_whole["City Group"] = df_whole["City Group"].map({"Other":0, "Big Cities":1})

#Typeを数値に変換 FC -> 0, IL -> 1, DT -> 2, MB -> 3
df_whole["Type"] = df_whole["Type"].map({"FC":0, "IL":1, "DT":2, "MB":3})

#再びtrainとtestに分割
df_train = df_whole.iloc[:train.shape[0]]
df_test = df_whole.iloc[train.shape[0]:]

In [ ]:
df_train.columns

In [ ]:
#学習にう特徴量を取得
df_train_columns = [col for col in df_train.columns if col not in ["Id", "Open Date"]]

In [ ]:
search_params = {
    'n_estimators'      : [i for i in range(1,31)],
    'max_features'      : [0.1],
    'random_state'      : [i for i in range(280,321)],
    'n_jobs'            : [-1],
    'min_samples_split' : [3],
    'max_depth'         : [i for i in range(10,41)]
}


rf = GridSearchCV(
    RandomForestRegressor(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)

In [ ]:
rf.fit(df_train[df_train_columns], revenue)

In [ ]:
print(rf.best_estimator_)

In [ ]:
prediction = rf.predict(df_test[df_train_columns])

In [ ]:
submission = pd.DataFrame({"Id":df_test.Id, "Prediction":prediction})
submission.to_csv("TFI_submission.csv", index=False)